In [100]:
import pandas as pd
import numpy as np
import warnings
import datetime
warnings.filterwarnings('ignore')
data = pd.read_csv(r'bank-information.csv')
data.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [101]:
data = data.replace('unknown',np.nan)
print(data.shape)
data=data.drop(['poutcome','contact','day','month'],axis=1)
data=data.dropna()
data.dtypes

(45211, 17)


age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
duration      int64
campaign      int64
pdays         int64
previous      int64
y            object
dtype: object

one-hot job marital default housing loan y  
minmaxscaler age balance duration campaign   
code education pdays previous 

In [102]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [103]:
data = pd.get_dummies(data,columns=['job','marital','default','housing','loan'])
data.sample(2)

,age,education,balance,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,job_unemployed,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes
334,45,tertiary,24598,313,3,-1,0,no,0,0,...,0,1,0,0,1,0,0,1,1,0
32107,51,secondary,421,139,2,-1,0,no,0,0,...,0,0,1,0,1,0,0,1,1,0


In [104]:
code_colnm = ['education','pdays','previous','y']
for i in code_colnm:
    data[i] = LabelEncoder().fit_transform(data[[i]])
data.sample(2)

,age,education,balance,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,job_unemployed,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes
36208,35,0,189,95,1,0,0,0,1,0,...,0,0,1,0,1,0,0,1,1,0
33427,35,1,2179,207,1,0,0,0,0,0,...,0,0,1,0,1,0,1,0,1,0


In [105]:
scalar_col = ['age','balance','duration','campaign']
for i in scalar_col:
    data[i] = MinMaxScaler().fit_transform(data[[i]])
data.sample(2)

,age,education,balance,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,job_unemployed,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes
2565,0.363636,0,0.075291,0.004067,0.0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,1,0
9607,0.350649,1,0.075037,0.024197,0.0,0,0,0,1,0,...,0,1,0,0,1,0,1,0,1,0


In [106]:
X = data.drop(['y'],axis=1)
Y = data[['y']]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.7,random_state=10)
x_train=x_train.values
x_test=x_test.values
y_train=y_train.values
y_test=y_test.values

In [107]:
import tensorflow as tf
import datetime
from tensorflow import keras
from keras import layers,Sequential,metrics


In [108]:
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
def pre(x,y):
    y = tf.one_hot(y,depth=2)
    y = tf.squeeze(y)
    return x,y
def pre1(x,y):
    y = tf.squeeze(y)
    return x,y
db_train = db_train.map(pre).shuffle(1000).batch(200)
db_test = db_test.map(pre1).shuffle(1000).batch(200)
db_iter = iter(db_train)
sample = next(db_iter)
sample[0].shape,sample[1].shape

(TensorShape([200, 27]), TensorShape([200, 2]))

In [109]:
model = Sequential([
layers.Dense(19,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.001)),
layers.Dropout(0.1),
layers.Dense(16,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.001)),
layers.Dropout(0.1),
layers.Dense(12,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.001)),
layers.Dropout(0.2),
layers.Dense(8,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.001)),
layers.Dropout(0.1),
layers.Dense(4,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.001)),
layers.Dense(2)
])
model.build(input_shape=[None,27])
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3,),loss=tf.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
# model.fit(db_train,validation_data=db_test,validation_freq=1,epochs=100)


In [110]:
opt = keras.optimizers.Adam(learning_rate=0.2)
for epoch in range(100):
    for step,(x,y) in enumerate(db_train):
        opt.learning_rate = 0.2*(100-epoch)/100
        with tf.GradientTape() as tade:
           logits = model(x)
           loss_mse = tf.reduce_mean(tf.losses.MSE(y,logits))
           loss_ca = tf.reduce_mean(tf.losses.categorical_crossentropy(y,logits,from_logits=True))
        grads = tade.gradient(loss_ca,model.trainable_variables)
        opt.apply_gradients(zip(grads,model.trainable_variables))
        if step % 100 == 0:
            print('step: ',step,'mse: ',float(loss_mse),'caco: ',float(loss_ca))
    total = 0
    total_correct = 0
    for x,y in db_test:
        logits = model(x)
        prob = tf.nn.softmax(logits,axis=1)
        pred = tf.argmax(prob,axis=1)
        pred = tf.cast(pred,tf.int32)
        correct = tf.equal(pred,y)
        correct = tf.cast(correct,tf.int32)
        correct = tf.reduce_sum(correct)
        total_correct+=int(correct)
        total+=x.shape[0]
    acc = total_correct/total
    print('epoch: ',epoch,'acc: ',acc,'learning_rate ',float(opt.learning_rate))
    print('epoch {} is end at {}!'.format(epoch,datetime.datetime.now()))


step:  0 mse:  61.53388595581055 caco:  3.7287709712982178
step:  100 mse:  0.8630151152610779 caco:  0.38979050517082214
epoch:  0 acc:  0.8860163605494675 learning_rate  0.20000000298023224
epoch 0 is end at 2022-08-07 08:56:39.634609!
step:  0 mse:  0.7837433815002441 caco:  0.3874007761478424
step:  100 mse:  0.9238194227218628 caco:  0.4029068350791931
epoch:  1 acc:  0.8860163605494675 learning_rate  0.1979999989271164
epoch 1 is end at 2022-08-07 08:56:41.961535!
step:  0 mse:  0.6077690124511719 caco:  0.3604830801486969
step:  100 mse:  0.7908602952957153 caco:  0.2922196090221405
epoch:  2 acc:  0.8860163605494675 learning_rate  0.19599999487400055
epoch 2 is end at 2022-08-07 08:56:44.299910!
step:  0 mse:  0.8677315711975098 caco:  0.4105049967765808
step:  100 mse:  0.826403796672821 caco:  0.3028567433357239
epoch:  3 acc:  0.8860163605494675 learning_rate  0.1940000057220459
epoch 3 is end at 2022-08-07 08:56:46.629239!
step:  0 mse:  0.6494258642196655 caco:  0.36813065

KeyboardInterrupt: 